In [ ]:
import requests
import json
import pandas

In [ ]:
!pip install newspaper3k

In [ ]:
import newspaper
from newspaper import Article
import spacy
from spacy.matcher import Matcher

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
def extract_article_from_url(url):
    a = Article(url)
    a.download()
    a.parse()
    return a.text, a.publish_date

In [ ]:
def sentence_giver(doc, start, end):
    s = start
    while str(doc[s]) != '.':
        s = s - 1
        if s < 0:
            break
    e = end
    while str(doc[e]) != '.':
        e = e + 1
        if e > len(doc) - 1:
            break
    return str(doc[s+1:e])

In [ ]:
def location_finder(doc):
    locations = []
    sentences = []
    matcher = Matcher(nlp.vocab)
    pattern = [{"ENT_TYPE": "GPE"}]
    matcher.add("Location", [pattern])
    matches = matcher(doc)
    for match_id, start, end in matches:
        sentence = sentence_giver(doc, start, end)
        value = l_detail_filter(sentence)
        if value == 1:
            sentences.append(sentence)
            locations.append(doc[start:end])
    return locations, sentences

In [ ]:
def l_detail_filter(data):
    data = nlp(data)
    matcher = Matcher(nlp.vocab)
    pattern1 = [{"LOWER": "demonstrators"}]
    matcher.add("L_Filter", [pattern1])
    matches = matcher(data)
    if len(matches) == 0:
        return 0
    else:
        return 1

In [ ]:
def number_finder(doc):
    numbers = []
    sentences = []
    matcher = Matcher(nlp.vocab)
    pattern = [{"LIKE_NUM": True}]
    matcher.add("Number", [pattern])
    matches = matcher(doc)
    for match_id, start, end in matches:
        sentence = sentence_giver(doc, start, end)
        value = n_detail_filter(sentence)
        if value == 1:
            sentences.append(sentence)
            numbers.append(str(doc[start:end]))
    return numbers, sentences

In [ ]:
def n_detail_filter(data):
    data = nlp(data)
    matcher = Matcher(nlp.vocab)
    pattern1 = [{"LOWER": "held"}, {"LOWER": "in"}, {"LOWER" : "detention"}]
    matcher.add("N_Filter", [pattern1])
    matches = matcher(data)
    if len(matches) == 0:
        return 0
    else:
        return 1

In [ ]:
def risk_analyser(n_people, threshold1, threshold2):
    if n_people[0] > threshold1:
        if threshold2 > n_people[0]:
            return 'Medium Risk'
        else:
            return 'High Risk'
    else:
        return 'Low Risk'

In [ ]:
def solver(doc):
    n_people, n_s = number_finder(doc)
    print(n_people)
    print(n_s)
    l_places, l_s = location_finder(doc)
    print(l_places)
    print(l_s)

In [ ]:
link = 'https://www.ndtv.com/world-news/11-dead-as-myanmar-protesters-fight-troops-with-handmade-guns-firebombs-2409266'
content, date = extract_article_from_url(link)
doc = nlp(content)
print(doc)
final = solver(doc)

Anti-coup demonstrators in Myanmar fought back with handmade guns and firebombs

Anti-coup demonstrators in Myanmar fought back with handmade guns and firebombs against a crackdown by security forces in a town in the northwest but at least 11 of the protesters were killed, domestic media reported on Thursday.

Initially, six truckloads of troops were deployed to quell protesters in the town of Taze, the Myanmar Now and Irrawaddy news outlets said. When the protesters fought back with handmade guns, knives and firebombs, five more truckloads of troops were brought in.

Fighting continued into Thursday morning and at least 11 protesters were killed and about 20 wounded, the media said. There was no word of any casualties among the soldiers.

That would take the toll of civilians killed by security forces to over 600 since the junta seized power from the elected government of Aung San Suu Kyi on Feb. 1, according to the Assistance Association for Political Prisoners (AAPP). It had a toll 